# CS 178 Project
## Import part

In [1]:
import math, os, sys
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from skimage.transform import resize
# Example: Load and preprocess images
# Assuming 'images' is a list of images and 'labels' is the corresponding list of labels
images_resized = [resize(image, (64, 64)).flatten() for image in images] # Resize and flatten
X_train, X_test, y_train, y_test = train_test_split(images_resized, labels, test_size=0.2, random_state=42)

# Train SVM
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

from sklearn.neighbors import KNeighborsClassifier

# KNN model
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Evaluate the model
y_pred = knn.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
from sklearn.ensemble import RandomForestClassifier

# Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


ModuleNotFoundError: No module named 'skimage'